In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from martini._RBmapping import RBMartiniMapping
from prostat.nn.martinibackmapper import MartiniBackmapper

In [3]:
### Either give pdb ###
pdb_filename = "chignolin.pdb"
### Or topology + traj ###
top_filename = "chignolin.psf"
traj_filename = "chignolin.xtc"

In [4]:
mapping = RBMartiniMapping("mapping")
mapping.map(top_filename, traj_filename)

In [5]:
import yaml
from pathlib import Path

import torch
import numpy as np

from prostat.nn.martinibackmapper import MartiniBackmapper

In [6]:
################################
### Build Martini Backmapper ###
################################

conf = {
    'device': 'cuda:0'
}
# path = f'{conf["name"]}_{locality}_{desired_stride}.pth'

model = MartiniBackmapper(
    n_atoms = mapping.n_atoms,
    n_beads = mapping.n_beads,
    bead_size = mapping.bead_size,
    bead2atom_idcs_mask=mapping.bead2atom_idcs_mask,
    bead2atom_idcs_generator=mapping.bead2atom_idcs_generator,
).to(conf["device"])
# try:
#     # model.load_state_dict(torch.load(path))
#     print(f'Model weights file {path} loaded!')
# except:
#     print(f'Model weights file {path} is missing')
#########################
#########################
#########################

In [7]:
P = torch.from_numpy(mapping._bead_positions).float().to(conf["device"])
p1 = torch.from_numpy(mapping._bead_v1).float().to(conf["device"])
p2 = torch.from_numpy(mapping._bead_v2).float().to(conf["device"])

x, rot_axes = model(
    P=P,
    p1=p1,
    p2=p2,
)

In [ ]:
config_file = "config/chignolin.yaml"
conf = yaml.safe_load(Path(config_file).read_text())

dataset = load_dataset(conf)

In [ ]:
dataset['bead_types']

In [ ]:
train_autoencoder(model, dataset, conf)

In [ ]:
pos_train = torch.from_numpy(dataset['pos_test']).double().to(conf['device'])
pos_recon, pos_beads, p1, p2, st, rot_axes, normalized_weights, contribution_beads = model(pos_train)

In [ ]:
torch.save(model.state_dict(), path)

In [ ]:
pos_recon, minimized_pos_recon, pos_beads, p1, p2, p12 = test_autoencoder(model, dataset, conf)

In [ ]:
nth = 100
pos1 = dataset['pos_test'][nth]
pos2 = pos_recon[nth].cpu().detach().numpy()

plot(
    nth,
    pos1,
    pos2,
    pos_beads,
    p1,
    p2,
    dataset,
    model.locality,
    model.desired_stride,
    model.encoder_lin.weights[..., 0],
    bond_idcs=dataset['bond_idcs']
)

In [ ]:
##################################
### Plot Dihedral Distribution ###
##################################
plot_dihedral_distribution(dataset, pos_recon, minimized_pos_recon)
##################################
##################################
##################################